<a href="https://colab.research.google.com/github/bahadirbesirkestane/Staj/blob/main/Parallel_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paralel Processing
Paralel Processing, görevin aynı bilgisayardaki birden fazla işlemcide aynı anda yürütüldüğü bir çalışma modudur. Amaç genel işlem süresini azaltmaktır.


In [2]:
import multiprocessing as mp
print("İşlemci sayısı: ", mp.cpu_count())

İşlemci sayısı:  2


Aynı anda çalıştırabileceğiniz maksimum işlem sayısı, bilgisayarınızdaki işlemci sayısına bağlıdır.

### Senkron ve Asenkron Yürütme nedir
Senkron yürütme, süreçlerin başlatıldığı sırayla tamamlanmasıdır. Bu, ilgili işlemler bitene kadar ana programın kilitlenmesiyle gerçekleştirilir.

Asenkron, kilitleme içermez. Sonuç olarak, sonuçların sırası karışabilir ancak genellikle daha hızlı tamamlanır.

In [5]:
# Her satırda belirli bir aralık arasında kaç sayı bulunduğunu sayın

import numpy as np
from time import time

# Prepare data
np.random.RandomState(100)
arr = np.random.randint(0, 10, size=[200000, 5])
data = arr.tolist()
data[:5]

[[9, 4, 7, 2, 5],
 [1, 0, 8, 5, 9],
 [3, 7, 4, 5, 4],
 [6, 7, 7, 9, 0],
 [9, 8, 3, 7, 0]]

>Paralelleştirme Olmadan Çözüm

In [7]:
def howmany_within_range(row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return count

results = []
for row in data:
  results.append(howmany_within_range(row, minimum=4, maximum=8))

print(results[:10])


[3, 2, 4, 3, 2, 5, 3, 1, 2, 5]


Herhangi bir işlemi paralelleştirmenin genel yolu, birden çok kez çalıştırılması gereken belirli bir işlevi alıp farklı işlemcilerde paralel olarak çalışmasını sağlamaktır.

### Pool.apply() kullanarak paralelleştirme

In [8]:
import multiprocessing as mp

# multiprocessing.Pool() başlatılması
pool = mp.Pool(mp.cpu_count())

# pool.apply nin değişkene uyarlanması
results = [pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data]

# pool un kapatılması
pool.close()

print(results[:10])

[3, 2, 4, 3, 2, 5, 3, 1, 2, 5]


###  Pool.map() kullanarak paralelleştirme

In [9]:
import multiprocessing as mp

# Redefine, with only 1 mandatory argument.
def howmany_within_range_rowonly(row, minimum=4, maximum=8):
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return count

pool = mp.Pool(mp.cpu_count())

results = pool.map(howmany_within_range_rowonly, [row for row in data])

pool.close()

print(results[:10])

[3, 2, 4, 3, 2, 5, 3, 1, 2, 5]


### Pool.starmap() kullanarak paralelleştirme

In [10]:
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())

results = pool.starmap(howmany_within_range, [(row, 4, 8) for row in data])

pool.close()

print(results[:10])

[3, 2, 4, 3, 2, 5, 3, 1, 2, 5]


## Asenkron Parallel Processing
Paralel olarak eşzamansız olarak yürütmenize olanak tanır; yani bir sonraki işlem, başlangıç sırasına bakılmaksızın bir önceki işlem biter bitmez başlayabilir.

Sonuç olarak sonucun girdiyle aynı sırada olacağının garantisi yoktur.

###  Pool.apply_async() ile Paralelleşitirme
Hesaplanan sonuçların nasıl saklanması gerektiğini söyleyen bir geri çağırma işlevi sağlamanız gerekmesi dışında application()'a çok benzer.

In [11]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

# Yinemele numarası tanımlanması
def howmany_within_range2(i, row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return (i, count)


# Çıktıyı "result"depişkeninde toplamak için geri çağırma işlevi
def collect_result(result):
    global results
    results.append(result)


# Paralelleştirmek için döngü
for i, row in enumerate(data):
    pool.apply_async(howmany_within_range2, args=(i, row, 4, 8), callback=collect_result)

# Havuzu kapat ve tüm işlemlerin tamamlanmasına izin ver
pool.close()
pool.join()  # Kuyruktaki tüm işlemler bitene kadar sonraki kod satırının yürütülmesini erteler.

# Step 5: Sonuçları sırala
results.sort(key=lambda x: x[0])
results_final = [r for i, r in results]

print(results_final[:10])

Bir geri çağırma işlevi sağlamadan application_async() işlevini kullanmak mümkündür.

Yalnızca bu, bir geri arama sağlamazsanız, her işlemden hesaplanan çıktı değerlerini içeren pool.ApplyResult nesnelerinin bir listesini alırsınız.

In [12]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

# application_async() işlevini çağırın
result_objects = [pool.apply_async(howmany_within_range2, args=(i, row, 4, 8)) for i, row in enumerate(data)]

# result_objects,pool.ApplyResult nesnelerinin bir listesidir
results = [r.get()[1] for r in result_objects]

pool.close()
pool.join()
print(results[:10])

[3, 2, 4, 3, 2, 5, 3, 1, 2, 5]


### Pool.starmap_async() ile Paralelleşitirme

In [13]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

results = pool.starmap_async(howmany_within_range2, [(i, row, 4, 8) for i, row in enumerate(data)]).get()

# map() de bunun yerine "howmany_within_range_rowonly" kullanın
# results =pool.map_async(howmany_within_range_rowonly, [verideki satır için satır]).get()

pool.close()
print(results[:10])

[(0, 3), (1, 2), (2, 4), (3, 3), (4, 2), (5, 5), (6, 3), (7, 1), (8, 2), (9, 5)]


## Pandas DataFrame'i Paralelleştirme
Bir DataFrame'in paralelleştirilmesi söz konusu olduğunda, paralelleştirilecek işlevin bir giriş parametresi olarak alınmasını sağlayabilirsiniz;
* veri çerçevesinin bir satırı
* veri çerçevesinin bir sütunu
* tüm veri çerçevesinin kendisi

In [14]:
import numpy as np
import pandas as pd
import multiprocessing as mp

df = pd.DataFrame(np.random.randint(3, 10, size=[5, 2]))
print(df.head())

   0  1
0  9  6
1  5  5
2  9  8
3  7  3
4  9  9


Bir veri çerçevemiz var. Her satıra hipotenüs fonksiyonunu uygulayalım, ancak aynı anda 4 işlemi çalıştıralım.

name=False ayarını yaparak, veri çerçevesinin her satırını basit bir demet olarak hipotenüs fonksiyonuna geçirmiş olursunuz.

In [19]:
# Satır bazında paralelleştirme
def hypotenuse(row):
    return round(row[1]**2 + row[2]**2, 2)**0.5

with mp.Pool(4) as pool:
    result = pool.imap(hypotenuse, df.itertuples(name=False), chunksize=10)
    output = [round(x, 2) for x in result]

print(output)

[9.43, 5.83, 5.0, 5.66, 11.4]

In [21]:
# Sütün Bazında Paralelleştirme
def sum_of_squares(column):
    return sum([i**2 for i in column[1]])

with mp.Pool(2) as pool:
    result = pool.imap(sum_of_squares, df.iteritems(), chunksize=10)
    output = [x for x in result]

print(output)

[317, 215]


### Pandas Dataframe, NumPy Array vb. kabul eden bir işlevi paralel hale getirmek.

In [25]:
import numpy as np
import pandas as pd
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool

df = pd.DataFrame(np.random.randint(3, 10, size=[500, 2]))

def func(df):
    return df.shape

cores=mp.cpu_count()

df_split = np.array_split(df, cores, axis=0)

# çoklu işlem havuzunu oluştur
pool = Pool(cores)

# havuzdaki her df'ye işlevi eşleyerek DataFrame'i işleyin
df_out = np.vstack(pool.map(func, df_split))

# havuzu kapat ve joinel
pool.close()
pool.join()
pool.clear()

**GPU Paralelleştirme :** Grafik işlem birimleri (GPU'lar), paralel işlemeye yönelik olarak optimize edilmiştir. Özellikle büyük veri işleme ve derin öğrenme gibi uygulamalarda GPU'lar kullanılır.

GPU ve CPU farkının bir matris çarpımıyla örneği:

>CPU

In [44]:
import tensorflow as tf
import time

# GPU cihazlarına erişimi devre dışı bırak
tf.config.experimental.set_visible_devices([], 'GPU')

#  MAtrisleri Oluştur
matrix_size = 1000
matrix_a = tf.random.normal((matrix_size, matrix_size))
matrix_b = tf.random.normal((matrix_size, matrix_size))

# Matris çarpımının CPU üzerinde hesaplanması
start_time = time.time()
result = tf.matmul(matrix_a, matrix_b)
end_time = time.time()

print("Matris çarpımı süresi (CPU):", end_time - start_time, "saniye")



Matris çarpımı süresi (CPU): 0.27014899253845215 saniye


>GPU

In [39]:
import tensorflow as tf
import time

# GPU cihazlarına erişimi etkinleştir
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# MAtrisleri Oluştur
matrix_size = 1000
matrix_a = tf.random.normal((matrix_size, matrix_size))
matrix_b = tf.random.normal((matrix_size, matrix_size))

# Matris çarpımının GPU üzerinde hesaplanması
start_time = time.time()
result = tf.matmul(matrix_a, matrix_b)
end_time = time.time()

print("Matris çarpımı süresi (GPU):", end_time - start_time, "saniye")



Matris çarpımı süresi (GPU): 0.06027626991271973 saniye
